## Практическая задача: Медицинский чат-бот с семантическим поиском

**Цель:** Создать интеллектуальную систему, которая может находить релевантную медицинскую информацию в базе знаний и отвечать на вопросы пользователей.

Дописать код для создания чат-бота, который:

1. **Индексирует** медицинские документы в векторной базе
2. **Находит** наиболее релевантные документы по запросу пользователя  
3. **Формирует** осмысленные ответы на основе найденной информации



In [1]:
pip install groq

In [2]:
import numpy as np
from groq import Groq
from sentence_transformers import SentenceTransformer

In [3]:
class MedicalChatBot:
    def __init__(self, documents):
        self.documents = documents
        self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.doc_embeddings = self.embedding_model.encode(documents, convert_to_numpy=True)
        self.groq_client = Groq(api_key="gsk_DY----zRf")

    def search_similar_documents(self, query, n_results=3):
        query_emb = self.embedding_model.encode(query, convert_to_numpy=True)
        query_norm = np.linalg.norm(query_emb)
        doc_norms = np.linalg.norm(self.doc_embeddings, axis=1)
        similarities = np.dot(self.doc_embeddings, query_emb) / (doc_norms * query_norm)
        top_indices = np.argsort(similarities)[::-1][:n_results]
        return [{"text": self.documents[i], "score": float(similarities[i])} for i in top_indices]

    def generate_answer(self, query, context_documents):
        if not context_documents or context_documents[0]["score"] < 0.25:
            return "Извините, я не нашёл достаточно релевантной информации по вашему запросу."
        context = "\n".join([doc["text"] for doc in context_documents])
        prompt = (
            f"Ты — медицинский помощник. Ответь на вопрос строго по предоставленному контексту.\n\n"
            f"Контекст:\n{context}\n\n"
            f"Вопрос: {query}\n\n"
            f"Ответ:"
        )
        response = self.groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=300
        )
        return response.choices[0].message.content.strip()

    def chat(self, user_query):
        docs = self.search_similar_documents(user_query, n_results=3)
        return self.generate_answer(user_query, docs)

# БАЗА ЗНАНИЙ ДЛЯ ТЕСТИРОВАНИЯ
medical_documents = [
    "Артериальная гипертензия - это стойкое повышение артериального давления выше 140/90 мм рт.ст.",
    "Диагностика гипертонии включает регулярные измерения давления, ЭКГ и анализы крови.",
    "Лечение гипертензии начинают с изменения образа жизни: снижение веса, ограничение соли, физическая активность.",
    "При неэффективности немедикаментозной терапии назначают антигипертензивные препараты: ингибиторы АПФ, бета-блокаторы, диуретики.",
    "Сахарный диабет 2 типа характеризуется инсулинорезистентностью и относительной инсулиновой недостаточностью.",
    "Диагностика диабета включает измерение уровня глюкозы крови натощак и тест на толерантность к глюкозе.",
    "Основные симптомы диабета: повышенная жажда, частое мочеиспускание, усталость, медленное заживление ран.",
    "Лечение диабета 2 типа включает диету, физические упражнения, метформин и при необходимости инсулин.",
    "Бронхиальная астма - хроническое воспалительное заболевание дыхательных путей.",
    "Симптомы астмы: одышка, свистящее дыхание, кашель, чувство стеснения в груди.",
    "Для лечения астмы используются ингаляционные кортикостероиды и бронходилататоры.",
    "Остеохондроз - дегенеративное поражение хряща и подлежащей кости.",
    "Симптомы остеохондроза: боль в пораженном отделе позвоночника, ограничение подвижности.",
    "Лечение остеохондроза включает ЛФК, физиотерапию, противовоспалительные препараты."
]



In [4]:
# ТЕСТОВЫЕ ЗАПРОСЫ
test_queries = [
    "Какие симптомы у диабета?",
    "Как лечить высокое давление?",
    "Что такое бронхиальная астма?",
    "Какие препараты используются при гипертензии?",
    "Как диагностируют остеохондроз?"
]

In [5]:
# Пробуем найти инфу
bot = MedicalChatBot(medical_documents)
print(bot.chat("Как лечить гипертонию?"))
print(bot.chat("Как диагностируют остеохондроз?"))
print(bot.chat("Что такое бронхиальная астма?"))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


К сожалению, в предоставленном контексте нет информации о лечении гипертонии. Контекст содержит информацию о диагностике диабета, лечении остеохондроза и характеристике сахарного диабета 2 типа, но не упоминает гипертонию.
В контексте не предоставлена информация о диагностике остеохондроза.
Бронхиальная астма - хроническое воспалительное заболевание дыхательных путей.


Домашнее задание:

- Добавить функцию для поиска с помощью FAISS
    - Функция возвращает результаты от 3х разных эмбеддинговых моделей
- Добавить PersistentClient к текущей реализации ChromaDB
    - Функция возвращает результат поиска и адрес временного хранилища базы данных
- Добавить любую кастомную модель к реализации ChromaDB (для Client)
    - Функция возвращает результат работы этой модели  

Образец вывода:

```
ЗАПРОС: ...
БОТ:
    - Ответ FAISS (модель 1: ...): ...
    - Ответ FAISS (модель 2: ...): ...
    - Ответ FAISS (модель 3: ...): ...
    - Ответ ChromaDB (Client, модель по умолчанию): ...
    - Ответ ChromaDB (Client, кастомная модель ...): ...
    - Ответ ChromaDB (PersistentClient, модель по умолчанию, имя временного файла: ...): ...
```

*Достаточно протестировать модель на одном любом запросе*

In [6]:
!pip install -q sentence-transformers faiss-cpu chromadb
!pip install -U chromadb

In [7]:
import tempfile
from sentence_transformers import SentenceTransformer
import faiss
import chromadb
from chromadb.utils import embedding_functions


# FAISS
def search_with_faiss(query, documents, model_name, n_results=3):
    model = SentenceTransformer(model_name)
    doc_embeddings = model.encode(documents, convert_to_numpy=True)
    query_emb = model.encode([query], convert_to_numpy=True)

    index = faiss.IndexFlatIP(doc_embeddings.shape[1])  # inner product = cosine after norm
    faiss.normalize_L2(doc_embeddings)
    faiss.normalize_L2(query_emb)
    index.add(doc_embeddings)

    scores, indices = index.search(query_emb, n_results)
    results = []
    for i in range(len(indices[0])):
        idx = indices[0][i]
        results.append(documents[idx])
    return " ".join(results)

# ChromaDB
def search_with_chroma_client(documents, query, embedding_fn=None, persist_directory=None):
    if persist_directory:
        client = chromadb.PersistentClient(path=persist_directory)
    else:
        client = chromadb.Client()

    # Удалим коллекцию, если существует (для чистоты)
    try:
        client.delete_collection("medical")
    except:
        pass

    collection = client.create_collection(
        name="medical",
        embedding_function=embedding_fn
    )

    collection.add(
        documents=documents,
        ids=[str(i) for i in range(len(documents))]
    )

    results = collection.query(query_texts=[query], n_results=3)
    return " ".join(results['documents'][0])

#  Генерация финального ответа
def generate_final_answer(query):
    #  FAISS: 3 модели
    faiss_1 = search_with_faiss(query, medical_documents, "all-MiniLM-L6-v2")
    faiss_2 = search_with_faiss(query, medical_documents, "all-mpnet-base-v2")
    faiss_3 = search_with_faiss(query, medical_documents, "paraphrase-multilingual-MiniLM-L12-v2")

    # ChromaDB: обычный Client + модель по умолчанию
    chroma_default = search_with_chroma_client(medical_documents, query)

    # ChromaDB: Client + кастомная модель
    custom_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="paraphrase-multilingual-MiniLM-L12-v2"
    )
    chroma_custom = search_with_chroma_client(medical_documents, query, embedding_fn=custom_ef)

    #  ChromaDB: PersistentClient + временная папка
    temp_dir = tempfile.mkdtemp()
    chroma_persistent = search_with_chroma_client(medical_documents, query, persist_directory=temp_dir)

    # Формируем ответ
    answer = f"""ЗАПРОС: {query}
БОТ:
    - Ответ FAISS (модель 1: all-MiniLM-L6-v2): {faiss_1}
    - Ответ FAISS (модель 2: all-mpnet-base-v2): {faiss_2}
    - Ответ FAISS (модель 3: paraphrase-multilingual-MiniLM-L12-v2): {faiss_3}
    - Ответ ChromaDB (Client, модель по умолчанию): {chroma_default}
    - Ответ ChromaDB (Client, кастомная модель paraphrase-multilingual-MiniLM-L12-v2): {chroma_custom}
    - Ответ ChromaDB (PersistentClient, модель по умолчанию, имя временного файла: {temp_dir}): {chroma_persistent}"""

    return answer

In [8]:
# Пример запроса
query = "Как лечить гипертонию?"
result = generate_final_answer(query)
print(result)

ЗАПРОС: Как лечить гипертонию?
БОТ:
    - Ответ FAISS (модель 1: all-MiniLM-L6-v2): Диагностика диабета включает измерение уровня глюкозы крови натощак и тест на толерантность к глюкозе. Лечение остеохондроза включает ЛФК, физиотерапию, противовоспалительные препараты. Сахарный диабет 2 типа характеризуется инсулинорезистентностью и относительной инсулиновой недостаточностью.
    - Ответ FAISS (модель 2: all-mpnet-base-v2): Лечение гипертензии начинают с изменения образа жизни: снижение веса, ограничение соли, физическая активность. Лечение остеохондроза включает ЛФК, физиотерапию, противовоспалительные препараты. Для лечения астмы используются ингаляционные кортикостероиды и бронходилататоры.
    - Ответ FAISS (модель 3: paraphrase-multilingual-MiniLM-L12-v2): Лечение гипертензии начинают с изменения образа жизни: снижение веса, ограничение соли, физическая активность. Диагностика гипертонии включает регулярные измерения давления, ЭКГ и анализы крови. Артериальная гипертензия - это ст